In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
from scipy import optimize as opt

import pandas as pd
import xarray as xr

import netCDF4 as nc4

import shutil
import datetime

In [2]:
def gauss(t,a,b,c):
    y = a*np.exp(-((t-b)**2.)/(2*c**2))
    return(y)

def gaussderiv(t,a,b,c):
    y = (a*(b-t)*np.exp(-((b-t)**2.)/(2*c**2.)))/(c**2.)
    return(y)

In [3]:
mapfile = xr.open_dataset('areacella_fx_CESM2_1pctCO2_r1i1p1f1_gn.nc')
areas = mapfile['areacella'].sum()
global_gridcell_integrated_area = float(areas.data)

/Users/cdkoven/miniconda2/envs/py3/lib/python3.7/site-packages/xarray/conventions.py:498: SerializationWarning: variable 'areacella' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


In [4]:
fname_base = 'emissions-cmip6_CO2_anthro_surface_175001-201512_fv_0.9x1.25_c20181011.nc'

datestring = str(datetime.date.today().year * 10000 + datetime.date.today().month * 100 + datetime.date.today().day)
fname_new = 'emissions_idealized_edrivencdr_000101-030012_fv_0.9x1.25_cdk_'+datestring+'.nc'

shutil.copyfile(fname_base, fname_new)

'emissions_idealized_edrivencdr_000101-030012_fv_0.9x1.25_cdk_20220214.nc'

In [5]:
file = nc4.Dataset(fname_new, 'r+')


In [6]:
time_firstyear = file.variables['time'][0:12]
time_bnds_firstyear = file.variables['time_bnds'][0:12,:]

In [7]:
nyears_out = 300
ts_out = nyears_out * 12
year_out = np.floor(np.arange(ts_out) / 12)
month_out = np.tile(time_firstyear,nyears_out)
date_out = year_out*365 + month_out

In [8]:
month_out_bnds = np.tile(time_bnds_firstyear,(nyears_out,1))
#month_out_bnds.shape
date_out_bnds = np.tile(year_out, (2,1)).transpose()*365 + month_out_bnds


In [9]:
file.variables['time'][:] = date_out

In [10]:
file.variables['time_bnds'][:] = date_out_bnds

In [11]:
#file.variables['CO2_flux']

In [12]:
t = date_out / 365.
y = gaussderiv(t,1004,150,45)

In [13]:
y

masked_array(data=[0.28830733631190597, 0.2899080342782799,
                   0.2914628152895189, ..., -0.29162407328474627,
                   -0.289961525499998, -0.2883605671425226],
             mask=[False, False, False, ..., False, False, False],
       fill_value=1e+20)

In [14]:
IM = len(file.variables['lon'][:])
JM = len(file.variables['lat'][:])


In [15]:
co2fluxdata = np.tile(y * 1.e12 / (global_gridcell_integrated_area * 86400. * 365.), (IM,JM,1))  # IM,JM))

In [16]:
co2fluxdata.transpose().shape

(3600, 192, 288)

In [17]:
file.variables['CO2_flux'][:].shape

(3600, 192, 288)

In [18]:
file.variables['CO2_flux'][:] = co2fluxdata.transpose()

In [19]:
file.variables['time'].units = 'days since 0001-01-01 00:00:00'

In [20]:
file.close()

In [21]:
y.cumsum()[3599]

0.020479102268602667

In [22]:
y.cumsum().argmax()

1799

In [23]:
y.argmax()

1260